In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical




In [ ]:
import glob

In [ ]:
# Load datasets from a folder
path = 'cleandata/'  # specify your folder path
all_files = glob.glob(path + "*.csv")  # finds all csv files in the folder

# Read each CSV file and concatenate them into one DataFrame
data = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

# Combine columns into a single text per row, handling missing values
todo_texts = data.apply(lambda x: ' '.join(x.dropna().values.tolist()), axis=1)

In [ ]:
todo_texts

0           Call name to discuss project details at time
1                        Water the plants in the morning
2           Grocery shopping at the local market at time
3                   Cancel gym membership over the phone
4        Prepare presentation slides for Mondays meeting
                              ...                       
19325                      Implement new morning routine
19326                        Attend friend's art exhibit
19327                               Purchase houseplants
19328                           File important documents
19329                            Back up smartphone data
Length: 19330, dtype: object

In [ ]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(todo_texts)
sequences = tokenizer.texts_to_sequences(todo_texts)

# Create input sequences
input_sequences = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        input_sequences.append(sequence[:i+1])

max_sequence_len = max(len(x) for x in input_sequences)
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))


In [ ]:
max_sequence_len

42

In [ ]:
len(input_sequences)

106805

In [ ]:
# Predictors and label
predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
label = to_categorical(label, num_classes=len(tokenizer.word_index) + 1)

# Model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=10, input_length=max_sequence_len - 1),
    LSTM(100),
    Dense(len(tokenizer.word_index) + 1, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(predictors, label, epochs=50, verbose=1)



Epoch 1/50
3338/3338 [==============================] - 146s 44ms/step - loss: 5.3498 - accuracy: 0.1295
Epoch 2/50
3338/3338 [==============================] - 143s 43ms/step - loss: 4.5043 - accuracy: 0.2141
Epoch 3/50
3338/3338 [==============================] - 143s 43ms/step - loss: 3.9925 - accuracy: 0.2732
Epoch 4/50
3338/3338 [==============================] - 143s 43ms/step - loss: 3.6264 - accuracy: 0.3160
Epoch 5/50
3338/3338 [==============================] - 143s 43ms/step - loss: 3.3492 - accuracy: 0.3471
Epoch 6/50
3338/3338 [==============================] - 147s 44ms/step - loss: 3.1348 - accuracy: 0.3725
Epoch 7/50
3338/3338 [==============================] - 143s 43ms/step - loss: 2.9649 - accuracy: 0.3915
Epoch 8/50
3338/3338 [==============================] - 144s 43ms/step - loss: 2.8264 - accuracy: 0.4070
Epoch 9/50
3338/3338 [==============================] - 143s 43ms/step - loss: 2.7102 - accuracy: 0.4211
Epoch 10/50
3338/3338 [==============================] 

In [ ]:
model.fit(predictors, label, epochs=80, verbose=1)

Epoch 1/80
3338/3338 [==============================] - 146s 44ms/step - loss: 1.6193 - accuracy: 0.5853
Epoch 2/80
3338/3338 [==============================] - 144s 43ms/step - loss: 1.6134 - accuracy: 0.5854
Epoch 3/80
3338/3338 [==============================] - 144s 43ms/step - loss: 1.6061 - accuracy: 0.5878
Epoch 4/80
3338/3338 [==============================] - 143s 43ms/step - loss: 1.6007 - accuracy: 0.5889
Epoch 5/80
3338/3338 [==============================] - 143s 43ms/step - loss: 1.5941 - accuracy: 0.5894
Epoch 6/80
3338/3338 [==============================] - 143s 43ms/step - loss: 1.5884 - accuracy: 0.5909
Epoch 7/80
3338/3338 [==============================] - 143s 43ms/step - loss: 1.5828 - accuracy: 0.5904
Epoch 8/80
3338/3338 [==============================] - 143s 43ms/step - loss: 1.5759 - accuracy: 0.5933
Epoch 9/80
3338/3338 [==============================] - 143s 43ms/step - loss: 1.5704 - accuracy: 0.5953
Epoch 10/80
3338/3338 [==============================] 

In [ ]:
# Prediction function
def predict_next_words(model, tokenizer, text, num_words=3):
    for _ in range(num_words):
        sequence = tokenizer.texts_to_sequences([text])[0]
        padded_sequence = pad_sequences([sequence], maxlen=max_sequence_len-1, padding='pre')
        predictions = model.predict(padded_sequence)[0]
        predicted_word_indices = np.argsort(predictions)[-num_words:]
        for idx in predicted_word_indices:
            output_word = tokenizer.index_word.get(idx, '')
            print(output_word)

In [ ]:
predict_next_words(model,tokenizer,"have a meeting with")

1/1 [==============================] - 0s 27ms/step
potential
a
name
1/1 [==============================] - 0s 36ms/step
potential
a
name
1/1 [==============================] - 0s 33ms/step
potential
a
name


In [ ]:
def predict_next_words(model, tokenizer, text, num_words=15):
    current_text = text
    for _ in range(num_words):
        sequence = tokenizer.texts_to_sequences([current_text])[0]
        padded_sequence = pad_sequences([sequence], maxlen=max_sequence_len-1, padding='pre')
        predictions = model.predict(padded_sequence)[0]
        next_word_index = np.argmax(predictions)
        next_word = tokenizer.index_word.get(next_word_index, '')
        current_text += ' ' + next_word
        print(next_word)


In [ ]:
predict_next_words(model,tokenizer,"buy")

1/1 [==============================] - 0s 28ms/step
a
1/1 [==============================] - 0s 33ms/step
new
1/1 [==============================] - 0s 30ms/step
charger
1/1 [==============================] - 0s 28ms/step
for
1/1 [==============================] - 0s 26ms/step
the
1/1 [==============================] - 0s 31ms/step
laptop
1/1 [==============================] - 0s 29ms/step
next
1/1 [==============================] - 0s 27ms/step
month
1/1 [==============================] - 0s 29ms/step
and
1/1 [==============================] - 0s 27ms/step
adjust
1/1 [==============================] - 0s 26ms/step
on
1/1 [==============================] - 0s 29ms/step
app
1/1 [==============================] - 0s 38ms/step
for
1/1 [==============================] - 0s 28ms/step
next
1/1 [==============================] - 0s 28ms/step
month


In [ ]:
model.save('my_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# 保存模型为 Keras 推荐的格式
model.save('my_model.keras')


In [ ]:
import pickle

with open('tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
